In [1]:
import sys  # Remove in production - KTZ

sys.path.append("..")  # Remove in production - KTZ
import pandas as pd
import datetime
from PolygonTickData.Helper import Helper
from CalculateETFArbitrage.LoadEtfHoldings import LoadHoldingsdata
from CalculateETFArbitrage.GatherData import DataApi
import numpy as np

# Jupyter Notebook to display all the columns in a df
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [20]:
def EtfMover(df=None, columnName=None):
    df=round(df,4)
    arr=df.abs().values.argsort(1)[:, -10:][:, ::-1]
    pos=0
    Change1=[]
    Change2=[]
    Change3=[]
    Change4=[]
    Change5=[]
    Change6=[]
    Change7=[]
    Change8=[]
    Change9=[]
    Change10=[]
    for i in arr:
        top=df.iloc[pos,].to_dict()
        l=[(k,v) for k,v in top.items()]
        Change1.append(l[i[0]])
        Change2.append(l[i[1]])
        Change3.append(l[i[2]])
        Change4.append(l[i[3]])
        Change5.append(l[i[4]])
        Change6.append(l[i[5]])
        Change7.append(l[i[6]])
        Change8.append(l[i[7]])
        Change9.append(l[i[8]])
        Change10.append(l[i[9]])
        pos+=1
    result=pd.DataFrame([Change1,Change2,Change3,
                         Change4,Change5,Change6,
                         Change7,Change8,Change9,
                         Change10]).T
    result.index=df.index
    result.columns=[columnName+str(i) for i in range(1,11)]
    return result

In [21]:
etfname = 'IYW'
date = '2020-03-17'

etfData = LoadHoldingsdata().LoadHoldingsAndClean(etfname=etfname, fundholdingsdate=date)
allData = DataApi(etfname=etfname, date=date, etfData=etfData)

ETF object
2020-03-17 00:00:00
Fetching Data
-------0.9006609916687012-------
Fetching Data
-------4.656590938568115-------


In [22]:
helperObj = Helper()
allData.tradesDataDf['Time'] = allData.tradesDataDf['Time'].apply(lambda x: helperObj.getHumanTime(ts=x, divideby=1000))
tradePricesDFMinutes = allData.tradesDataDf.groupby([allData.tradesDataDf['Time'], allData.tradesDataDf['Symbol']])['Trade Price']
tradePricesDFMinutes = tradePricesDFMinutes.first().unstack(level=1)

priceforNAVfilling = allData.openPriceData.set_index('Symbol').T.to_dict('records')[0]

In [23]:
marketStartTime = datetime.datetime.strptime('13:29:00', "%H:%M:%S").time()
marketEndTime = datetime.datetime.strptime('20:00:00', "%H:%M:%S").time()

mask = (tradePricesDFMinutes.index.time >= marketStartTime) & (tradePricesDFMinutes.index.time <= marketEndTime)
tradePricesDFMinutes=tradePricesDFMinutes[mask]
tradePricesDFMinutes=tradePricesDFMinutes.ffill(axis=0)
tradePricesDFMinutes = tradePricesDFMinutes.fillna(priceforNAVfilling)

etfprice = tradePricesDFMinutes[etfname]
tradePricesDFMinutes = tradePricesDFMinutes.pct_change().dropna() * 100

etfpricechange = tradePricesDFMinutes[etfname]
del tradePricesDFMinutes[etfname]

In [24]:
allData.quotesDataDf['Time'] = allData.quotesDataDf['Time'].apply(lambda x: helperObj.getHumanTime(ts=x, divideby=1000000000))
allData.quotesDataDf['Time'] = allData.quotesDataDf['Time'].map(lambda x: x.replace(second=0))
allData.quotesDataDf = allData.quotesDataDf[allData.quotesDataDf['Bid Size'] != 0]
allData.quotesDataDf = allData.quotesDataDf[allData.quotesDataDf['Ask Size'] != 0]
allData.quotesDataDf['Total Bid Ask Size'] = allData.quotesDataDf['Ask Size'] + allData.quotesDataDf['Bid Size']
allData.quotesDataDf['Spread'] = allData.quotesDataDf['Ask Price'] - allData.quotesDataDf['Bid Price']
allData.quotesDataDf['MidPrice'] = (allData.quotesDataDf['Ask Price'] + allData.quotesDataDf['Bid Price']) / 2
quotesSpreadsMinutes = allData.quotesDataDf.groupby('Time')['Spread'].mean()

In [ ]:
del allData

In [17]:
netassetvaluereturn = tradePricesDFMinutes.assign(**etfData.getETFWeights()).mul(tradePricesDFMinutes).sum(axis=1)

ds = pd.concat([etfprice, etfpricechange, netassetvaluereturn, quotesSpreadsMinutes], axis=1).dropna()
ds.columns = ['ETF Price', 'ETF Change Price %', 'Net Asset Value Change%', 'ETF Trading Spread in $']
ds['Arbitrage in $'] = (ds['ETF Change Price %'] - ds['Net Asset Value Change%']) * ds['ETF Price'] / 100
ds['Arbitrage in $'] = abs(ds['Arbitrage in $'])
ds['Flag'] = 0
ds.loc[(ds['Arbitrage in $'] > ds['ETF Trading Spread in $']) & ds['ETF Trading Spread in $'] != 0, 'Flag'] = 111

holdingsChange = EtfMover(df=tradePricesDFMinutes, columnName='Change%')
etfMoverholdings = EtfMover(df=tradePricesDFMinutes.assign(**etfData.getETFWeights()).mul(tradePricesDFMinutes).dropna(axis=1), columnName='ETFMover%')

ds=pd.concat([ds,etfMoverholdings,holdingsChange],axis=1)

In [26]:
quotesSpreadsMinutes

Time
2020-03-17 11:01:00    39.435000
2020-03-17 12:49:00    8.036667 
2020-03-17 13:01:00    6.668333 
2020-03-17 13:29:00    3.893000 
2020-03-17 13:30:00    0.934222 
2020-03-17 13:31:00    0.961864 
2020-03-17 13:32:00    0.975596 
2020-03-17 13:33:00    1.036650 
2020-03-17 13:34:00    0.694879 
2020-03-17 13:35:00    0.609477 
2020-03-17 13:36:00    0.565143 
2020-03-17 13:37:00    0.598123 
2020-03-17 13:38:00    0.702170 
2020-03-17 13:39:00    0.669975 
2020-03-17 13:40:00    0.500000 
2020-03-17 13:41:00    0.582621 
2020-03-17 13:42:00    0.568998 
2020-03-17 13:43:00    0.580187 
2020-03-17 13:44:00    0.502617 
2020-03-17 13:45:00    0.644732 
2020-03-17 13:46:00    0.678178 
2020-03-17 13:47:00    0.627377 
2020-03-17 13:48:00    0.519943 
2020-03-17 13:49:00    0.488592 
2020-03-17 13:50:00    0.428484 
2020-03-17 13:51:00    0.414812 
2020-03-17 13:52:00    0.420328 
2020-03-17 13:53:00    0.306837 
2020-03-17 13:54:00    0.343706 
2020-03-17 13:55:00    0.431859 
2020-

In [2]:
! pip install --upgrade matplotlib

     |████████████████████████████████| 12.5MB 459kB/s eta 0:00:01     |██████████████████████▍         | 8.7MB 24.3MB/s eta 0:00:01     |██████████████████████▋         | 8.8MB 24.3MB/s eta 0:00:01     |████████████████████████████████| 12.4MB 459kB/s eta 0:00:01
  Found existing installation: matplotlib 3.1.1
    Uninstalling matplotlib-3.1.1:
      Successfully uninstalled matplotlib-3.1.1
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from mpl_finance import candlestick_ohlc

ModuleNotFoundError: No module named 'mpl_finance'

In [5]:
! pip install mplfinance

You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
! pip install matplotlib.finance

ERROR: Could not find a version that satisfies the requirement matplotlib.finance (from versions: none)
ERROR: No matching distribution found for matplotlib.finance
You should consider upgrading via the 'pip install --upgrade pip' command.
